In [1]:
# conda install -c rapidsai -c nvidia -c conda-forge dask-cuda cudatoolkit=11.6
# nvidia-smi
from sklearn.model_selection import train_test_split, KFold
import dask.dataframe as dd
import dask.multiprocessing
import dask
import numpy as np
from dask import delayed
import dask_image.imread
import dask_image.ndfilters
import dask_image.ndmeasure
import cv2
from glob import glob
import json
import dask.array as da
import time
import xgboost as xgb
import pandas as pd
import os
from dask import compute
from sklearn.metrics import f1_score
# f1_score(y_true, y_pred, average=[‘micro’, ‘macro’, ‘samples’,’weighted’ 중 하나 선택])
import tensorflow as tf

@delayed
def padding(data, array_len, col_len):
    pad = np.zeros((array_len, col_len))
    length = min(array_len, len(data))
    
    pad[:length] = data[:length]
    return pad

@delayed
def img_resize(img):
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255
    return img

path = os.getcwd()
csv_features = ['내부 온도 1 평균', '내부 온도 1 최고', '내부 온도 1 최저', '내부 습도 1 평균', '내부 습도 1 최고', 
                '내부 습도 1 최저', '내부 이슬점 평균', '내부 이슬점 최고', '내부 이슬점 최저']

# from dask.distributed import Client, LocalCluster
# # client.close()
# cluster = LocalCluster()
# client = Client(cluster)
# print(cluster,client)
# client.close()
# http://localhost:8787/status

# from dask_cuda import LocalCUDACluster
# from dask.distributed import Client
# client.close()
# # http://127.0.0.1:34497/status
# cluster = LocalCUDACluster()
# client = Client(cluster)
# print(cluster,client)

2022-01-28 14:50:22.693607: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2022-01-28 14:50:26.025927: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-28 14:50:26.030595: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-28 14:50:27.696553: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:927] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-01-28 14:50:27.696596: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3060 Laptop GPU computeCapability: 8.6
coreClock: 1.282GHz coreCount: 30 deviceMemorySize: 6.00GiB deviceMemoryBandwidth: 268.26GiB/s
2022-01-28 14:50:27.696621: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-01-28 14:50:27.697878: I tensorflow/stream_exec

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4267788769424119566,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 4932619488
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 10473751795663331274
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"]

In [3]:
#making_label_set

crop = {'1':'딸기','2':'토마토','3':'파프리카','4':'오이','5':'고추','6':'시설포도'}
disease = {'1':{'a1':'딸기잿빛곰팡이병','a2':'딸기흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '2':{'a5':'토마토흰가루병','a6':'토마토잿빛곰팡이병','b2':'열과','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '3':{'a9':'파프리카흰가루병','a10':'파프리카잘록병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '4':{'a3':'오이노균병','a4':'오이흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '5':{'a7':'고추탄저병','a8':'고추흰가루병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '6':{'a11':'시설포도탄저병','a12':'시설포도노균병','b4':'일소피해','b5':'축과병'}}
risk = {'0':'정상','1':'초기','2':'중기','3':'말기'}


#ensemble_labler
label_description = {}
for key, value in disease.items():
    label_description[f'{key}_00_0'] = f'{crop[key]}_정상'
    for disease_code in value:
        for risk_code in risk:
            label = f'{key}_{disease_code}_{risk_code}'
            label_description[label] = f'{crop[key]}_{disease[key][disease_code]}_{risk[risk_code]}'

global ensemble_label_encoder
ensemble_label_encoder = {key:idx for idx, key in enumerate(label_description)}
ensemble_label_decoder = {val:key for key, val in ensemble_label_encoder.items()}

#crop_labler
crop_label_description = {}
for key, value in disease.items():
    crop_label_description[f'{key}'] = f'{crop[key]}'
    
global crop_label_encoder
crop_label_encoder = {key:idx for idx, key in enumerate(crop_label_description)}
crop_label_decoder = {val:key for key, val in crop_label_encoder.items()}


#disease_labler
disease_label_description = {}
for key, value in disease.items():
    disease_label_description[f'00'] = "정상"
    for disease_code,value in value.items():
        label = f'{disease_code}'
        disease_label_description[label] = f'{value}'

global disease_label_encoder
disease_label_encoder = {key:idx for idx, key in enumerate(disease_label_description)}
disease_label_decoder = {val:key for key, val in disease_label_encoder.items()}


#risk_labler
risk_label_description = {}
for key, value in risk.items():
    label = f'{key}'
    risk_label_description[label] = f'{value}'

global risk_label_encoder
risk_label_encoder = {key:idx for idx, key in enumerate(risk_label_description)}
risk_label_decoder = {val:key for key, val in risk_label_encoder.items()}





In [4]:
crop_label_description

{'1': '딸기', '2': '토마토', '3': '파프리카', '4': '오이', '5': '고추', '6': '시설포도'}

In [5]:
risk_label_description

{'0': '정상', '1': '초기', '2': '중기', '3': '말기'}

In [6]:
disease_label_description

{'00': '정상',
 'a1': '딸기잿빛곰팡이병',
 'a2': '딸기흰가루병',
 'b1': '냉해피해',
 'b6': '다량원소결핍 (N)',
 'b7': '다량원소결핍 (P)',
 'b8': '다량원소결핍 (K)',
 'a5': '토마토흰가루병',
 'a6': '토마토잿빛곰팡이병',
 'b2': '열과',
 'b3': '칼슘결핍',
 'a9': '파프리카흰가루병',
 'a10': '파프리카잘록병',
 'a3': '오이노균병',
 'a4': '오이흰가루병',
 'a7': '고추탄저병',
 'a8': '고추흰가루병',
 'a11': '시설포도탄저병',
 'a12': '시설포도노균병',
 'b4': '일소피해',
 'b5': '축과병'}

In [46]:
label_description

{'1_00_0': '딸기_정상',
 '1_a1_0': '딸기_딸기잿빛곰팡이병_정상',
 '1_a1_1': '딸기_딸기잿빛곰팡이병_초기',
 '1_a1_2': '딸기_딸기잿빛곰팡이병_중기',
 '1_a1_3': '딸기_딸기잿빛곰팡이병_말기',
 '1_a2_0': '딸기_딸기흰가루병_정상',
 '1_a2_1': '딸기_딸기흰가루병_초기',
 '1_a2_2': '딸기_딸기흰가루병_중기',
 '1_a2_3': '딸기_딸기흰가루병_말기',
 '1_b1_0': '딸기_냉해피해_정상',
 '1_b1_1': '딸기_냉해피해_초기',
 '1_b1_2': '딸기_냉해피해_중기',
 '1_b1_3': '딸기_냉해피해_말기',
 '1_b6_0': '딸기_다량원소결핍 (N)_정상',
 '1_b6_1': '딸기_다량원소결핍 (N)_초기',
 '1_b6_2': '딸기_다량원소결핍 (N)_중기',
 '1_b6_3': '딸기_다량원소결핍 (N)_말기',
 '1_b7_0': '딸기_다량원소결핍 (P)_정상',
 '1_b7_1': '딸기_다량원소결핍 (P)_초기',
 '1_b7_2': '딸기_다량원소결핍 (P)_중기',
 '1_b7_3': '딸기_다량원소결핍 (P)_말기',
 '1_b8_0': '딸기_다량원소결핍 (K)_정상',
 '1_b8_1': '딸기_다량원소결핍 (K)_초기',
 '1_b8_2': '딸기_다량원소결핍 (K)_중기',
 '1_b8_3': '딸기_다량원소결핍 (K)_말기',
 '2_00_0': '토마토_정상',
 '2_a5_0': '토마토_토마토흰가루병_정상',
 '2_a5_1': '토마토_토마토흰가루병_초기',
 '2_a5_2': '토마토_토마토흰가루병_중기',
 '2_a5_3': '토마토_토마토흰가루병_말기',
 '2_a6_0': '토마토_토마토잿빛곰팡이병_정상',
 '2_a6_1': '토마토_토마토잿빛곰팡이병_초기',
 '2_a6_2': '토마토_토마토잿빛곰팡이병_중기',
 '2_a6_3': '토마토_토마토잿빛곰팡이병_말기',
 '2_b2_0': '토마토_열과_정상',

In [7]:
@delayed
def label_encoding(label):
    global ensemble_label_encoder
    encoded_label = ensemble_label_encoder[label]
    return encoded_label

@delayed
def label_encoding_crop(label):
    global crop_label_encoder
    encoded_label = crop_label_encoder[label]
    return encoded_label

@delayed
def label_encoding_disease(label):
    global disease_label_encoder
    encoded_label = disease_label_encoder[label]
    return encoded_label

@delayed
def label_encoding_risk(label):
    global risk_label_encoder
    encoded_label = risk_label_encoder[label]
    return encoded_label

def getlable(jsonpath,type):
    if type == "ensemble":
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)

        crop = json_file['annotations']['crop']
        disease = json_file['annotations']['disease']
        risk = json_file['annotations']['risk']
        label = f'{crop}_{disease}_{risk}'
        return label

    elif type == "crop":
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)
        crop = json_file['annotations']['crop']
        label = f'{crop}'
        return label

    elif type == "dc":
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)
        disease = json_file['annotations']['disease']
        label = f'{disease}'
        return label

    elif type == "risk":  
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)
        risk = json_file['annotations']['risk']
        label = f'{risk}'
        return label

    
def get_label_list(labelpath_list):
    labelarr = np.array([])
    labelarr_crop = np.array([])
    labelarr_dc = np.array([])
    labelarr_risk = np.array([])


    # labelarr = da.array([])
    for ind,json_path in enumerate(labelpath_list):
        # label = label_encoding(getlable(json_path))
        # label = da.array(np.array(label_encoding(getlable(json_path))))
        label = np.array(label_encoding(getlable(json_path,"ensemble")))
        labelarr = np.append(labelarr,label)

        label_crop = np.array(label_encoding_crop(getlable(json_path,"crop")))
        labelarr_crop = np.append(labelarr_crop,label_crop)

        label_dc = np.array(label_encoding_disease(getlable(json_path,"dc")))
        labelarr_dc = np.append(labelarr_dc,label_dc)

        label_risk = np.array(label_encoding_risk(getlable(json_path,"risk")))
        labelarr_risk = np.append(labelarr_risk,label_risk)


    return labelarr, labelarr_crop, labelarr_dc, labelarr_risk

In [8]:
@delayed
def get_all_image(imgpath_list,image_size):
    imgarr = np.empty((0,image_size,image_size,3), float)
    for ind,img_path in enumerate(imgpath_list):
        imgarr = delayed(np.append)(imgarr,img, axis = 0)
    return imgarr

In [9]:
%%time
label_list = sorted(glob(os.path.join(path,"data","train","*","*.json")))
y_train,y_train_crop,y_train_dc,y_train_risk, = get_label_list(label_list)
# y_train = y_train.rechunk(5767)
results = dask.compute(*y_train)
label = np.array(results)
# print(label.shape)
results_crop = dask.compute(*y_train_crop)
label_crop = np.array(results_crop)
#
results_dc = dask.compute(*y_train_dc)
label_disease = np.array(results_dc)
#
results_risk = dask.compute(*y_train_risk)
label_risk = np.array(results_risk)
label.shape,label_crop.shape,label_disease.shape,label_risk.shape

CPU times: user 3.52 s, sys: 20 ms, total: 3.54 s
Wall time: 3.45 s


((5767,), (5767,), (5767,), (5767,))

In [10]:
%%time
@delayed
def imageresize(img):
    # img = dask.delayed(cv2.resize)(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    # img = dask.delayed(img.astype(np.float32)/255)  ##픽셀값을 0~1사이로 정규화
    # # img = np.transpose(img, (2,0,1))
    # return dask.delayed(img.reshape)(-1,224,224,3)
    img = cv2.resize(img, dsize=(100, 100), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255  ##픽셀값을 0~1사이로 정규화
    # img = np.transpose(img, (2,0,1))
    return da.array(img)
#making_img_set
row_img = dask_image.imread.imread(os.path.join(path,"data","train","*","*.jpg"))
train = [imageresize(img) for img in row_img]
train_x = da.array(dask.compute(*train))

CPU times: user 40 s, sys: 2.8 s, total: 42.8 s
Wall time: 13.3 s


In [61]:
train_x = train_x.reshape(5767,-1)
label = da.array(label)

In [72]:
X, X_test, y, y_test = train_test_split(train_x, label, test_size=0.2, shuffle= True,stratify=label)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.5, shuffle= True,stratify=y)

In [73]:
%%time
params = {"tree_method": "gpu_hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
        #   "single_precision_histogram" : True,
          }

start = time.time()
XGB_mod1 = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X_train[:,:1000],y_train[:])
# print(time.time()-start)
y_boosted = XGB_mod1.predict_proba(X_val[:,:1000])

XGB_mod2 = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(y_boosted,y_val[:])

y_pred_first = XGB_mod1.predict_proba(X_test[:,:1000])
y_pred = XGB_mod2.predict(y_pred_first)

/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/core.py:502: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  format(", ".join(args_msg)), FutureWarning
/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


CPU times: user 5min 18s, sys: 347 ms, total: 5min 18s
Wall time: 22.2 s


In [75]:
from sklearn import metrics

# Print the confusion matrix
print(metrics.confusion_matrix(y_test, y_pred))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test, y_pred, digits=3))

[[ 65   1   2  34   2   0   1   3   0   5   3  26   2   0   1   2   3  12
    0   0   0   0   0   0   0]
 [  5   3   0   6   0   0   0   0   0   0   0   9   0   0   0   1   0   5
    0   0   0   0   0   0   0]
 [  3   0   5  18   0   0   0   0   2   0   1   1   0   0   2   1   0   5
    0   0   0   0   0   0   0]
 [ 35   0   4 133   4   1   0   4   1   3   3  23   0   0   3   4   7  10
    0   0   0   0   0   0   0]
 [  3   0   1   8   4   0   0   0   1   4   0   5   0   0   2   2   0   0
    0   0   0   0   0   0   1]
 [  1   0   0   8   2   1   0   0   0   2   0   5   0   0   2   0   0   1
    0   0   0   0   0   0   0]
 [  0   0   0   4   0   0   0   0   1   0   1   1   0   0   0   0   0   0
    0   0   0   0   0   0   1]
 [  3   1   1  12   1   0   0   3   1   2   1   6   0   0   0   1   1   0
    0   0   0   0   0   0   0]
 [  4   0   0  19   1   1   0   0   0   0   1   2   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  6   0   0   8   0   1   0   1   0   2   0   9   0  

/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [47]:
# %%time
# if __name__ == "__main__":
#     with LocalCluster(dashboard_address=':3258') as cluster:
#         with Client(cluster) as client:
#             print(client)
#             # X and y must be Dask dataframes or arrays
#             params = {"verbosity": 0, 
#             "tree_method": "hist", 
#             "objective": 'multi:softmax',
#             "num_class":len(label_description),
#             }
#             dtrain = xgb.dask.DaskDMatrix(client, tempx,tempy)
#             out = xgb.dask.train(
#                 client,
#                 params,
#                 dtrain,
#                 num_boost_round=20,
#                 evals=[(dtrain, "train")],
#             )
#             preds2 = xgb.dask.predict(client,out,tempx_test).compute()

              

<Client: 'tcp://127.0.0.1:45861' processes=4 threads=16, memory=24.59 GiB>


[15:13:25] task [xgboost.dask]:tcp://127.0.0.1:40773 got new rank 0
[15:13:25] task [xgboost.dask]:tcp://127.0.0.1:43227 got new rank 1


[0]	train-mlogloss:2.98982
[1]	train-mlogloss:2.00949
[2]	train-mlogloss:1.58097
[3]	train-mlogloss:1.30929
[4]	train-mlogloss:1.09855
[5]	train-mlogloss:0.94797
[6]	train-mlogloss:0.81695
[7]	train-mlogloss:0.71333
[8]	train-mlogloss:0.62851
[9]	train-mlogloss:0.55313
[10]	train-mlogloss:0.49089
[11]	train-mlogloss:0.44403
[12]	train-mlogloss:0.40025
[13]	train-mlogloss:0.36272
[14]	train-mlogloss:0.32717
[15]	train-mlogloss:0.29757
[16]	train-mlogloss:0.26816
[17]	train-mlogloss:0.24655
[18]	train-mlogloss:0.22433
[19]	train-mlogloss:0.20494
CPU times: user 23.3 s, sys: 1.96 s, total: 25.3 s
Wall time: 2min 16s


In [49]:
from dask_cuda import LocalCUDACluster
# cluster = LocalCUDACluster()

In [50]:
# %%time
# if __name__ == "__main__":
#     with LocalCUDACluster(dashboard_address=':3258') as cluster:
#         with Client(cluster) as client:
#             print(client)
#             # X and y must be Dask dataframes or arrays
#             params = {"verbosity": 0, 
#             "tree_method": "hist", 
#             "objective": 'multi:softmax',
#             "num_class":len(label_description),
#             }
#             dtrain = xgb.dask.DaskDMatrix(client, tempx,tempy)
#             out2 = xgb.dask.train(
#                 client,
#                 params,
#                 dtrain,
#                 num_boost_round=20,
#                 evals=[(dtrain, "train")],
#             )
#             preds3 = xgb.dask.predict(client,out2,tempx_test).compute()

              

/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/dask_cuda/utils.py:216: UserWarning: Cannot get CPU affinity for device with index 0, setting default affinity
  % device_index
distributed.preloading - INFO - Import preload module: dask_cuda.initialize
Unable to start CUDA Context
Traceback (most recent call last):
  File "/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/dask_cuda/initialize.py", line 42, in _create_cuda_context
    ctx = has_cuda_context()
  File "/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/distributed/diagnostics/nvml.py", line 78, in has_cuda_context
    running_processes = pynvml.nvmlDeviceGetComputeRunningProcesses_v2(handle)
  File "/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/pynvml/nvml.py", line 2220, in nvmlDeviceGetComputeRunningProcesses_v2
    raise NVMLError(ret)
pynvml.nvml.NVMLError_Unknown: Unknown Error
distributed.worker - WARNING - Mismatched versions found

+-------------+-------------+------

<Client: 'tcp://127.0.0.1:44143' processes=1 threads=1, memory=24.59 GiB>


distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/distributed/protocol/core.py", line 112, in loads
    frames[0], object_hook=_decode_default, use_list=False, **msgpack_opts
  File "msgpack/_unpacker.pyx", line 195, in msgpack._cmsgpack.unpackb
  File "/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/distributed/protocol/core.py", line 104, in _decode_default
    sub_header, sub_frames, deserializers=deserializers
  File "/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/distributed/protocol/serialize.py", line 488, in merge_and_deserialize
    return deserialize(header, merged_frames, deserializers=deserializers)
  File "/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/distributed/protocol/serialize.py", line 417, in deserialize
    return loads(header, frames)
  File "/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/distrib

AssertionError: error